<a href="https://colab.research.google.com/github/aknip/Streamlit-Gradio/blob/main/myGPTlab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# myGPTlab

This app helps to process texts (longform content) with ChatGPT via API.

## Start:
**Run all cells - that's it.**
- The notebooks checks automatically, if an initial setup (with PIP etc.) is necessary. The setup status is saved in the file 'installation.done'
- You can force the setup by deleting this file or by going to Seciton "Setup and Configuration" and checking "inital_setup_mode". Afterwards uncheck "Setup and Configuration".

## Working with myGPTlab

## FAQ
### Was tun, wenn kryptische Fehlermeldungen erscheinen, wie z. B. "error: invalid group reference 11 at position 1"?
Den Input-Text neu anlegen bzw. konvertieren (z. b. per pandoc). Die Ursache kann Encoding sein.




Here we go

In [1]:
# @title
import os
import requests
# os.environ.pop('CREDS') # delete environment variable for debugging
if not os.getenv('CREDS'):
  #popup('Enter API Keys as JSON:')
  fname = 'aknip-colab-setup'
  url = 'https://bit.ly/' + fname
  r = requests.get(url)
  open(fname, 'wb').write(r.content)
  %run aknip-colab-setup
else:
  print("API Keys available in env 'CREDS'")

!pip install ipywidgets -q

Secrets (JSON string): ··········
Credentials stored in env 'CREDS'. Get values via: 
creds = json.loads(os.getenv('CREDS')) 
key = creds['OpenAI']['v1']['credential']
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.8 MB/s eta 0:00:00


In [2]:
# @title
import ipywidgets
from ipywidgets import Layout, Button, Box, VBox, HTML, Output, Textarea

# Info flex proportionally to the weight
info_1 = HTML(value='<div style="background-color: red; text-align: center"><b>Please wait for initial setup...</b></div>', layout=Layout(flex='1 1 auto', width='auto'))
items_info = [info_1]

# Header flex proportionally to the weight
header_1 = HTML(value='<div style="background-color: #ccc; text-align: center">Hello <b>World</b></div>', layout=Layout(flex='1 1 auto', width='auto'))
items_header = [header_1]

# Text flex proportionally to the weight
text_1 = HTML(value='Infotext', layout=Layout(flex='1 1 auto', width='25%'))
text_2 = HTML(value='Infotext jdskl fjkdslf jsdklfjslf', layout=Layout(flex='1 1 auto', width='25%'))
text_3 = HTML(value='Infotext jdskl fjkdslf jsdklfjslf', layout=Layout(flex='1 1 auto', width='25%'))
text_4 = HTML(value='Infotext jdskl fjkdslf jsdklfjslf', layout=Layout(flex='1 1 auto', width='25%'))
items_texts = [text_1, text_2, text_3, text_4]

# Textinput flex proportionally to the weight
input_1 = Textarea(placeholder='Enter input text', layout=Layout(flex='1 1 auto', width='25%', height='150px'), disabled=False)
input_2 = Textarea(placeholder='Enter input text', layout=Layout(flex='1 1 auto', width='25%', height='150px'), disabled=False)
input_3 = Textarea(placeholder='Enter input text', layout=Layout(flex='1 1 auto', width='25%', height='150px'), disabled=False)
input_4 = Textarea(placeholder='Enter input text', layout=Layout(flex='1 1 auto', width='25%', height='150px'), disabled=False)
items_inputs = [input_1, input_2, input_3, input_4]

# Buttons flex proportionally to the weight
button_1 = Button(description='Summarize Articles', layout=Layout(flex='1 1 0%', width='auto'), button_style='success')
button_2 = Button(description='Save Text', layout=Layout(flex='1 1 0%', width='auto'), button_style='success')
button_3 = Button(description='Split Text', layout=Layout(flex='1 1 0%', width='auto'), button_style='success')
button_4 = Button(description='Chat', layout=Layout(flex='1 1 0%', width='auto'), button_style='success')
items_buttons = [button_1, button_2, button_3, button_4]

# Output flex proportionally to the weight
output_1 = Output(layout=Layout(flex='1 1 auto', width='auto'))
items_output = [output_1]

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='90%')

box_info = Box(children=items_info, layout=box_layout)
box_header = Box(children=items_header, layout=box_layout)
box_texts = Box(children=items_texts, layout=box_layout)
box_inputs = Box(children=items_inputs, layout=box_layout)
box_buttons = Box(children=items_buttons, layout=box_layout)
box_output = Box(children=items_output, layout=box_layout)

def on_button_1_clicked(b):
    #with output_1:
    comment = 'Kommentar zum Projekt...'
    execute_project('Handelsblatt', 'gpt-3.5-turbo', '', promptlib_global, 'summary-multiple-articles', '1', 'length-max-dyn', '', 0 ,True, folders, input_1.value, comment)
button_1.on_click(on_button_1_clicked)

def on_button_2_clicked(b):
    output_1.clear_output()

button_2.on_click(on_button_2_clicked)

#button_1.layout.visibility = "hidden"
#button_2.layout.visibility = "hidden"
#button_3.layout.visibility = "hidden"
#button_4.layout.visibility = "hidden"

VBox([box_info, box_header, box_texts, box_inputs, box_buttons, box_output])


# Install

In [3]:
# @title Settings

# @markdown Default model
default_model = 'GPT-3.5' # @param ["GPT-3.5", "GPT-4"]

# @markdown Start Gradio webapp.
start_gradio_webapp = False # @param {type:"boolean"}

# @markdown Initial Setup Mode for pip install, fetch credentials etc.
initial_setup_mode = False # @param {type:"boolean"}

# @markdown Debug Mode for extensive logging.
debug_mode = True # @param {type:"boolean"}

# @markdown iOS Mode to develop helper functions, no Gradio.
# @markdown Useful for development on iOS, eg. with Carnets App
ios_mode = False # @param {type:"boolean"}

In [4]:
folders = {
    'audio': 'audio',
    'audio-chunks': 'audio/chunks',
    'transcript':'audio-transcript',
    'transcript-chunks': 'audio-transcript/chunks',
    'text-input': 'text-input',
    'text-input-backup': 'text-input-backup',
    'text-output': 'text-output',
    'text-output-logs': 'text-output-logs'
}

In [5]:
from ipywidgets import widgets
from IPython.display import Javascript, display, clear_output
notify_output = widgets.Output()
display(notify_output)
@notify_output.capture()
def popup(text):
    clear_output()
    display(Javascript("alert('{}')".format(text)))
#popup('Hello World!')

Output()

In [6]:
import os
if initial_setup_mode != True:
  if os.path.exists('installation.done'):
      initial_setup_mode = False
      print('No initial setup - forced by existing file "installation.done"')
  else:
    initial_setup_mode = True
    print('Starting automatic setup - forced by missing file "installation.done".\n\nEnter API Keys as JSON (in next notebook cell).')
    #popup('Starting automatic setup. Enter API Keys as JSON (in next notebook cell).')
else:
  print('Starting setup.\n\nEnter API Keys as JSON (in next notebook cell).')
  #popup('Starting setup. Enter API Keys as JSON (in next notebook cell).')

Starting automatic setup - forced by missing file "installation.done".

Enter API Keys as JSON (in next notebook cell).


In [7]:
creds = json.loads(os.getenv('CREDS'))
# openAI_key = creds['OpenAI']['v2']['credential']
# print(openAI_key)

In [8]:
if initial_setup_mode == True:
  !pip install openai==0.27.7 yt-dlp==2023.7.6 librosa==0.10.0.post2 pickle-mixin==1.0.2 langchain==0.0.225 PyPDF2==3.0.1 PyMuPDF==1.22.5 pypandoc==1.11 gradio -q
  %load_ext gradio
else:
  print('No initial setup.')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 5.5 MB/s eta 0:00:

In [9]:
if initial_setup_mode == True:
  f= open('installation.done','w+')
  f.close()
  print('Initial setup done. Application starting.')
  # popup('Initial setup done. Application starting.')
else:
  print('No initial setup.')

Initial setup done. Application starting.


In [10]:
import requests

def import_prompt_lib():
  fname = 'prompt_lib.ipynb'
  url = 'https://raw.githubusercontent.com/aknip/Streamlit-Gradio/main/' + fname
  r = requests.get(url)
  open(fname, 'wb').write(r.content)
  %run prompt_lib.ipynb

def import_helper_lib():
  fname = 'helper_lib.ipynb'
  url = 'https://raw.githubusercontent.com/aknip/Streamlit-Gradio/main/' + fname
  r = requests.get(url)
  open(fname, 'wb').write(r.content)
  %run helper_lib.ipynb

In [11]:
import_helper_lib()
import_prompt_lib()

In [12]:
print(promptlib_global)
print(testfunc(3,4))

{'summary-shortened': {'description': '', 'category': 'summarize', '1': {'note': 'Gleichzeitiges Einfügen/Formatieren von Headlines funktioniert nicht, lässt GPT alles umformulieren           ("Format headings and subheadings in markdown syntax.").           Test S4/HANA Kap. 4: Input 12.300 Wörter, GPT-3.5 Output: 4.500 Wörter (36%), GPT-4 Output: 3.400 Wörter (27%)', 'prompt': 'Paraphrase the following text and shorten each paragraph of the paraphrased text to maxiumum the half of its original length by focussing            on the key aspects. Write in the style and tone of a professional business analyst and use passive form. Be detailed and precise            and use every content detail of the text. Note that there may be typographical or grammatical errors in the passages due to the            text being fetched from a transcript of a talk or a speech.            {language_text} {length_text} Text:```{input_text}``` {toc_text}', 'lang-de': 'Write everything in German language.', 

In [ ]:
# @title
test_text1 = 'Die Photosynthese ist ein physiologischer Prozess zur Erzeugung energiereicher Biomoleküle aus energieärmeren Stoffen mit Hilfe\
  von Lichtenergie. Sie wird von Pflanzen, Algen und manchen Bakterien betrieben. Bei diesem biochemischen Vorgang wird Lichtenergie mit Hilfe\
  von lichtabsorbierenden Farbstoffen wie Chlorophyll in chemische Energie umgewandelt. Diese wird dann genutzt, um aus energiearmen\
  anorganischen Stoffen (vor allem Kohlenstoffdioxid (CO2) und Wasser (H2O)) energiereiche organische Verbindungen (vor allem Kohlenhydrate)\
  aufzubauen. Der genutzte Anteil der eingestrahlten Energie, nämlich der zum Aufbau der Assimilate verwendete Anteil, wird photosynthetische\
  Effizienz genannt. Soweit die energiereichen organischen Stoffe zu Bestandteilen des Lebewesens werden, bezeichnet man deren Synthese als\
  Assimilation. Man unterscheidet zwischen oxygener und anoxygener Photosynthese. Bei der oxygenen Photosynthese wird molekularer\
  Sauerstoff (O2) freigesetzt. Bei der anoxygenen Photosynthese, die nur von Bakterien betrieben wird, entstehen statt Sauerstoff andere\
  anorganische Stoffe, beispielsweise elementarer Schwefel (S). Die Photosynthese ist der einzige biochemische Prozess, bei dem\
  Lichtenergie, meistens Sonnenlicht, in chemisch gebundene Energie umgewandelt wird (Phototrophie). Indirekt sind auch fast alle\
  heterotrophen (nicht zur Photosynthese fähigen) Lebewesen von ihr abhängig, da sie der Photosynthese letztlich ihre Nahrung und auch\
  den zur Energiegewinnung mittels aerober Atmung benötigten Sauerstoff verdanken. Aus dem Sauerstoff entsteht außerdem die schützende\
  Ozonschicht der Erdatmosphäre.'
test_text2 = f'''
# The Potential of AI in Education

So, anyone who's been paying attention for the last few months has been seeing headlines like this, especially in education. The thesis has been students are going to be using chat GPT and other forms of AI to cheat, do their assignments, they're not going to learn, and it's going to completely undermine education as we know it.

Now what I'm going to argue today is not only are there ways to mitigate all of that, if we put the right guardrails, we do the right things, we can mitigate it, but I think we're at the cusp of using AI for probably the biggest positive transformation that education has ever seen. And the way we're going to do that is by giving every student on the planet an artificially intelligent but amazing personal tutor, and we're going to give every teacher on the planet an amazing, artificially intelligent teaching assistant.  And just to appreciate how big of a deal it would be to give everyone a personal tutor, I show you this clip from Benjamin Bloom's 1984 two-sigma study, or he called it the two-sigma problem.

The two-sigma comes from two standard deviations, sigma the symbol for standard deviation, and he had good data that showed that, look, a normal distribution, that's the one that you see in the traditional bell curve right in the middle, that's how the world kind of sorts itself out, that if you were to give personal one-to-one tutoring for students, that you could actually get a distribution that looks like that right, it says tutorial one-to-one with the asterisks, like that right distribution, a two standard deviation improvement. Just to put that in plain language, that could take your average student and turn them into an exceptional student, it can take your below average student and turn them into an above average student.

# The Challenge of Scaling Personalized Instruction

Now, the reason why he framed it as a problem was he said, well, this is all good, but how do you actually scale group instruction this way? How do you actually give it to everyone in an economic way? What I'm about to show you is, I think, the first moves towards doing that. Obviously, we've been trying to approximate it in some way at Khan Academy for over a decade now, but I think we're at the cusp of accelerating it dramatically.  I'm going to show you the early stages of what RAI, which we call Khan Migo, what it can now do, and maybe a little bit of where it is actually going.  This right over here is a traditional exercise that you or many of your children might have seen on Khan Academy, but what's new is that little bot thing at the right, and we'll start by seeing one of the very important safeguards, which is the conversation is recorded and viewable by your teacher.  It's moderated, actually, by a second AI, and also, it does not tell you the answer.  It is not a cheating tool.  Notice, when the student says, tell me the answer, it says, I'm your tutor.

What do you think is the next step for solving the problem? Now, if the student makes a mistake, and this will surprise people who think large language models are not good at mathematics, notice not only does it notice the mistake, it asks the student to explain their reasoning, but it's actually doing what I would say not just even an average tutor would do, but an excellent tutor would do. It's actually able to divine what is probably the misconception in that student's mind, that they probably didn't use the distributive properly.  Remember, we need to distribute the negative two to both the nine and the 2M inside of the parentheses.  This to me is a very, very, very big deal, and it's not just in math.  This is a computer programming exercise on Khan Academy where the student needs to make the clouds part, and so we can see the student starts defining a variable, left X minus minus.

# AI as a Super Tutor

It only made the left cloud part, but then they can ask a con amigo, what's going on? Why is only the left cloud moving? And it understands the code. It knows all the context of what the student is doing, and it understands that those ellipses are there to draw clouds, which I think is kind of mind-blowing, and it says, to make the right cloud move as well, try adding a line of code inside the draw function that increments the right X variable by one pixel in each frame.  Now, this one is maybe even more amazing, because we have a lot of math teachers.  We've all been trying to teach the world to code, but there aren't a lot of computing teachers out there, and what you just saw, even when I'm tutoring my kids when they're learning to code, I can't help them this well, this fast.  This is really going to be a super tutor.  And it's not just exercises.  It understands what you're watching.

It understands the context of your video. It can answer the age-old question, why do I need to learn this? And it asks, socratically, well, what do you care about? And let's say the student says, I want to be a professional athlete, and it says, well, learning about the size of cells, which is what this video is about, that could be really useful for understanding nutrition and how your body works, et cetera.  It can answer questions.  It can quiz you.  It can connect it to other ideas.  You can now ask as many questions of a video as you could ever dream of.  So another big shortage out there, I remember the high school I went to, the student-to-guidance counselor ratio was about 200 or 300 to 1.  A lot of the country, it's worse than that.  We can use Conmigo to give every student a guidance counselor, academic coach, career coach, life coach, which is exactly what you see right over here.  And this is all, you know, we launched this with the GPT-4 launch.  We have a few thousand people on this.  This isn't a fake demo.

§§§

# Conversations with Historical Figures and AI Debates

This is really in action. And then there's, you know, things that I think it would have been even harder, it would have been a little science fiction to do with even a traditional tutor.  There's a student, we run an online high school with Arizona State University called Khan World School.  And we have a student who attends that online school based in India, her name is Sanvi, and she was doing a report on the Great Gatsby.  And when she was reading the Great Gatsby, Jay Gatsby keeps looking at the green light off into the distance.  And she's like, well, why does he do that? She did some web searches, and people have obviously studied this and commented about the symbolism of that, but none of it was really resonating with her.  And then she realized that she had Conmigo and that she could talk to Jay Gatsby himself.

# Unlocking Learning Literature and AI Debates

And so, ah, splendid choice, old sport, I am now Jay Gatsby, the enigmatic millionaire from Escott Jarrell's classic. And so why do you keep staring at the green light? Ah, the green light, old sport.  It's a symbol of my dreams and desires, you see.  It's situated at the end of Daisy Buchanan's dock, across the bay from my mansion.  I gaze at it longingly as it represents my yearning for the past and my hope to reunite with Daisy, the love of my life.  And what was cool is Sanvi had said, I had this long conversation, she called him Mr.  Gatsby.  And at the end, she actually apologized for taking his time, which I thought was very polite of her.  But you can imagine, this unlocks learning literature, learning, you can talk to historical figures, we're even probably going to add an activity, you can talk to, like, the Mississippi River.  It brings things to life in ways that really were science fiction even six months or a year ago.  Students can get into debates with the AI.  And we've got, here's the students debating whether we should cancel student debt.  The student is against canceling student debt.  And we've gotten very clear feedback.  We've started running it at Con World School and our lab school that we have, Con Lab School.  The students, the high school students especially, they're saying, this is amazing to be able to fine tune my arguments without fearing judgment.

# Enhancing Language Arts with AI Writing Collaboration

It makes me that much more confident to kind of go into the classroom and really participate. And we all know that Socratic dialogue debate is a great way to learn.  But frankly, it's not out there for most students.  But now it can be accessible to hopefully everyone.  A lot of the narrative, we saw that in the headlines, has been it's going to do the writing for kids.  Kids are not going to learn to write.  But we are showing that there's ways that the AI doesn't write for you.  It writes with you.  So this is a little thing.  And my eight-year-old is addicted to this.  And he's not a kid that really liked writing before.  But where, you know, you could say, I want to write a horror story.  And it says, ooh, a horror story.  How spine-tingling and thrilling.  Let's dive into the world of eerie shadows and chilling mysteries.

And this is an activity where the student will write two sentences. And then the AI will write two sentences.  And so they collaborate together on a story.  The students write, Beatrice was a misunderstood ghost.  She wanted to make friends but kept scaring them by accident.  And the AI says, poor Beatrice, a lonely spirit yearning for companionship.  One day she stumbled upon an old abandoned mansion, et cetera, et cetera.  I encourage you all to, you know, hopefully one day try this.  This is surprisingly fun.  Now to even more directly hit this use case.  And what I'm about to show you, everything I showed you so far is actually already part of Calmigo.  What I'm about to show you, we haven't shown to anyone yet.

# Enhancing Reading Comprehension and Writing Skills with AI

This is a prototype. We hope to be able to launch it in the next few months.  But this is to directly use AI, use generative AI, to not undermine English and language arts but to actually enhance it in ways that we couldn't have even conceived of even a year ago.  This is reading comprehension.  This is the students reading Steve Jobs' famous speech at Stanford.  And then as they get to certain points, they can click on that little question.  And the AI will then, Socratically, almost like an oral exam, ask the student about things.  And the AI can highlight parts of the passage.  Why did the author use that word? What was their intent? Does it back up their argument? They can start to do stuff that, once again, we never had the capability to give everyone a tutor, everyone a writing coach, to actually dig into reading at this level.  And you could go on the other side of it.  We have a whole workflow that helps them write, helps them be a writing coach, draw an outline.  But once a student actually constructs a draft, and this is where they're constructing a draft, they can ask for feedback, once again, as you would expect from a good writing coach.  In this case, the student we'll say, let's say, does my evidence support my claim? And then the AI not only is able to give feedback, but it's able to highlight certain parts of the passage and says, you know, on this passage, this doesn't quite support your claim, but once again, Socratically says, can you tell us why? So it's pulling the student, it's making them a better writer, giving them far more feedback than they've ever been able to actually get before, and we think this is going to dramatically accelerate writing, not hurt it.

§§§

# Personalized Education for Teachers
Now, everything I've talked about so far is for the student, but we think this could be equally as powerful for the teacher to drive more personalized education and, frankly, save time and energy for themselves and for their students. So this is an American history exercise on Khan Academy.  It's a question about the Spanish-American war.  And at first, it's in student mode, and if you say, tell me the answer, it's not going to tell the answer, it's going to go into tutoring mode.  But that little toggle which teachers have access to, they can turn student mode off, and then it goes into teacher mode.  And what this does is, it turns into, you could view it as a teacher's guide on steroids.  Not only can it explain the answer, it can explain how you might want to teach it.

# Benefits for Teachers
It can help prepare the teacher for that material. It can help them create lesson plans, as you can see doing right there.  It'll eventually help them create progress reports, it'll help them eventually grade.  So once again, teachers spend about half their time with this type of activity, lesson planning, all of that energy can go back to them or go back to human interactions with their actual students.  So, you know, one point I want to make, these large language models are so powerful, there's a temptation to say, well, all these people are just going to slap them onto their websites, and it kind of turns the applications themselves into commodities.  And what I've got to tell you is, I kind of thought that's one of the reasons why I didn't sleep for two weeks when I first had access to GPT-4 back in August.

# Enhancing AI Tutoring
But we quickly realized that to actually make it magical, I think it's really important to make it magical, but we quickly realized that to actually make it magical, I think what you saw with Conmigo a little bit, it didn't interact with you the way that you see chat GPT interacting, it was a little bit more magical, it was more Socratic, it was clearly much better at math than what most people are used to thinking. And the reason is there was a lot of work behind the scenes to make that happen.  And I could go through the whole list of everything we've been working on, many, many people, for over six, seven months, to make it feel magical, but perhaps the most intellectually interesting one is we realized, and this was an idea from an open AI researcher, that we could dramatically improve its ability in math and its ability in tutoring if we allowed the AI to think before it speaks.

# The Future of AI and Education
So if you're tutoring someone and you immediately just start talking before you assess their math, you might not get it right. But if you construct thoughts for yourself, and what you see on the right there is an actual AI thought, something that it generates for itself but it does not share with the student, then its accuracy went up dramatically and its ability to be a world-class tutor went up dramatically.  And you can see it's talking to itself here.  It says, the student got a different answer than I did, but do not tell them they made a mistake.  Instead, ask them to explain how they got to that step.  So I'll just finish off.

# The Role of AI in Education and the Need for Positive Use Cases
Hopefully, what I've just shown you is just half of what we are working on, and we think this is just the very tip of the iceberg of where this can actually go. And I'm pretty convinced, which I wouldn't have been even a year ago, that we, together, have a chance of addressing the two-sigma problem and turning it into a two-sigma opportunity, dramatically accelerating education as we know it.  Now, just to take a step back at a meta-level, obviously, we heard a lot today, the debates on either side.  There's folks who take a more pessimistic view of AI.  They say, this is scary, there's all these dystopian scenarios.  We maybe want to slow down.  We want to pause.  On the other side, there are the more optimistic folks who say, well, we've gone through inflection points before.

We've gone through the Industrial Revolution. It was scary, but it all kind of worked out.  And what I'd argue right now is, I don't think this is like a flip of a coin or this is something where we'll just have to wait and see which way it turns out.  I think everyone here and beyond, we are active participants in this decision.  I'm pretty convinced that the first line of reasoning is actually almost a self-fulfilling prophecy, that if we act with fear and if we say, hey, we just got to stop doing this stuff, what's really going to happen is the rule followers might pause, might slow down, but the rule breakers, as Alexander mentioned, the totalitarian governments, the criminal organizations, they're only going to accelerate.

And that leads to what I am pretty convinced is the dystopian state, which is the good actors have worse AIs than the bad actors. But I'll also talk to the optimist a little bit.  I don't think that means that, oh yeah, then we should just relax and just hope for the best.  That might not happen either.  I think all of us together have to fight like hell to make sure that we put the guardrails, we put in, when the problems arise, reasonable regulations, but we fight like hell for the positive use cases.  Because very close to my heart, and obviously there's many potential positive use cases, perhaps the most powerful use case, and perhaps the most poetic use case, is if AI, artificial intelligence, can be used to enhance HI, human intelligence, human potential and human purpose.  Thank you.  Thank you.  Thank you.
'''

# Gradio App

In [14]:
def run_gradio_app():
  import gradio as gr

  # Theming
  theme = gr.themes.Default(
      primary_hue="slate" # , radius_size=gr.themes.Size(radius_sm="3px", radius_xs="2px", radius_xxs="1px")
  )
  # Styling: Change max width
  css = """
    .gradio-container {max-width: 800px!important}
    .vspacer1 {margin-top: 50px}
  """

  with gr.Blocks(theme=theme, css=css) as demo:

      gr.Markdown("# ChatGPTLab 2.0", elem_classes="vspacer1")
      gr.Markdown("### Optimizing your work with LLMs.")

      project_name = gr.Textbox(label="Project name")

      #
      # 1. Input Text
      #
      with gr.Tab("Input Text "):
        gr.Markdown("Please enter text")

        # Input text via UI
        gr.Markdown("### Input your text:")
        text_input = gr.Textbox(label="Enter text", placeholder="Your text here...", lines=10)
        text_output = gr.Textbox(label="Result")

        def text_save(text, proj_name):
          create_file_directory(proj_name, False)
          create_file_directory(proj_name + '/' +  folders['text-input'], False)
          f= open(proj_name + '/' +  folders['text-input'] + '/input_text.txt','w+')
          f.write(text)
          f.close()
          log_text = "Text saved."
          return log_text
        text_button = gr.Button("Save text")
        text_button.click(text_save, [text_input, project_name], text_output)

        gr.Markdown("")
        gr.Markdown("")

        # Input text via upload
        gr.Markdown("### Or upload your text:")
        upload_button = gr.UploadButton("Click to Upload a File", file_types=[".txt",".md"], file_count="single")
        file_output = gr.Textbox(label="Result")

        def upload_file(my_file, proj_name):
          create_file_directory(proj_name, False)
          create_file_directory(proj_name + '/' +  folders['text-input'], False)
          # copy to project directory
          full_upload_path = my_file.name
          just_the_filename = os.path.basename(full_upload_path)
          full_text_path = "./" + proj_name + '/' + folders['text-input'] + '/' + just_the_filename
          shutil.copyfile(full_upload_path, full_text_path)
          # check if file is empty
          f= open(full_text_path,'r')
          if f.mode == 'r': contents =f.read()
          f.close()
          log_text = just_the_filename + "\n"
          if len(contents) == 0:
            log_text = log_text + "Error: Upload file lengt 0 bytes"
          else:
            log_text = log_text + "Upload successful"
          return log_text
        upload_button.upload(upload_file, [upload_button, project_name], file_output)

      #
      # 2. Download full project
      #
      with gr.Tab("Download"):
        gr.Markdown("Download full project as ZIP file.")
        download_button = gr.Button("Download project")
        download_output = gr.File()

        def download_do(proj_name):
          full_text_path = "./" + proj_name
          shutil.make_archive('archive', 'zip', full_text_path)
          result = "Downloading " + full_text_path
          return "archive.zip"
        download_button.click(download_do, project_name, download_output)

      #
      # 3. xxx
      #
      with gr.Tab("Step 2"):
        gr.Markdown("Please select the optimization:")
        radio = gr.Radio(
          ["by headline", "by paragraph", "by §§§"], label="Text split method"
        )
        name = gr.Textbox(label="Name", placeholder="Enter text...")
        output = gr.Textbox(label="Output Box")
        greet_btn = gr.Button("Start", scale=0)
        def greet(name):
          result = "HALLO " + name + "!!!"
          return result
        greet_btn.click(fn=greet, inputs=name, outputs=output, api_name="greet")

  demo.launch(quiet=True, share=False, debug=debug_mode)

run_gradio_app()

<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.


# Test GPT

In [ ]:
tmp = split_text_by_separator(test_text1, '\n', '\n', 'fix-nothing', 200)
print(text_stats(test_text1))
print(len(tmp))

In [ ]:
# test promptlib and templates

import textwrap

prompt_obj = get_prompt_from_lib(promptlib_global, 'summary-bullets', '1')
# prompt_obj = get_prompt_from_lib(promptlib_global, 'summary-bullets', '1')
print(json.dumps(prompt_obj, sort_keys=True, indent=2))

input_text = 'This is the input text. Lorem ipsum.'

prompt_txt_final1 = build_prompt_from_template(input_text, prompt_obj, 'lang-de', 'length-max-dyn', 0.5, 'toc')
prompt_txt_final2 = build_prompt_from_template(input_text, prompt_obj, 'lang-de', 'length-max-fix', 0, '')
prompt_txt_final3 = build_prompt_from_template(input_text, prompt_obj, 'lang-de', '', 0, '')
print(textwrap.fill(prompt_txt_final1, 120))

#write_prompt_to_lib(prompt_obj)

#print(json.dumps(promptlib, sort_keys=True, indent=2))

# myprompt = "The given text is delimited by triple backticks. Summarize the current text to succint and clear bullet points of its contents.
#          The length of the summary must be 200 words maximum."
# myprompt = "The given text is delimited by triple backticks.
#           Summarize the current text to a maximum of 15 succint and clear bullet points of its contents."
# myprompt = myprompt + "The maximum number of words should be 300 words in total. "
# myprompt = myprompt + "Write everything in German language. " # this is optional !
# myprompt = myprompt + "```" + input_text + "```"

In [ ]:
info_1.layout.visibility = "hidden"
info_1.close()
del info_1
button_1.layout.visibility = "visible"
button_2.layout.visibility = "visible"
button_3.layout.visibility = "visible"
button_4.layout.visibility = "visible"
#text_1.value="yoyo"

# Workbench

In [ ]:
proj_name = 'S4HANA'
#create_file_directory(proj_name, True) # For dev: Initial folder structure
#create_file_directory(proj_name + '/' + folders['text-output'], True)
#create_file_directory(proj_name + '/' + folders['text-output-logs'], True)
#create_test_project(proj_name, '')

# Step 1
def step_1():
  model = 'gpt-3.5-turbo'
  prompt_name = 'summary-shortened'
  prompt_version = '1'
  prompt_comment = 'TODO: HIER EINE NOTIZ. HIER EINE NOTIZ. HIER EINE NOTIZ. HIER EINE NOTIZ.'
  prompt_obj = get_prompt_from_lib(promptlib_global, prompt_name, model, prompt_version)
  #print(json.dumps(prompt_obj, sort_keys=False, indent=2))
  result_file = execute_prompt({
      'model': model,
      'prompt-name': prompt_name,
      'prompt-version': prompt_version,
      'prompt-comment': prompt_comment,
      'loglevel': 0,
      'stop-after-step': 1,
      'proj-name': proj_name,
      'input-filepath': '', #'text-output',
      'input-filename': '', #'paraphrased.txt',
      'folders': folders,
      'prompt-obj': prompt_obj,
      'language': 'lang-de',
      'length-type': 'length-max-dyn',
      'length-max': 0.5,
      'toc-text': '',
      'result-filename': '01-paraphr-short.md'
  })
  print(result_file)
  a, b = result_file
  print(a)
  print(b)

# Step 2
def step_2():
  model = 'gpt-3.5-turbo'
  prompt_name = 'create-subheadlines'
  prompt_version = '1'
  prompt_obj = get_prompt_from_lib(promptlib_work, prompt_name, model, prompt_version)
  #print(json.dumps(prompt_obj, sort_keys=False, indent=2))
  result_file = execute_prompt({
      'model': model,
      'prompt-name': prompt_name,
      'prompt-version': prompt_version,
      'loglevel': 0,
      'stop-after-step': 3,
      'proj-name': proj_name,
      'input-filepath': 'text-output',
      'input-filename': '01-paraphr-short_gpt-3.md',
      'folders': folders,
      'prompt-obj': prompt_obj,
      'language': 'lang-same',
      'length-type': '',
      'length-max': 0,
      'toc-text': '',
      'result-filename': '01-paraphr-short-headlines.md'
  })

# Step 3
def step_3():
  model = 'gpt-3.5-turbo'
  prompt_name = 'summary-shortened'
  prompt_version = '1'
  prompt_obj = get_prompt_from_lib(promptlib_work, prompt_name, model, prompt_version)
  #print(json.dumps(prompt_obj, sort_keys=False, indent=2))
  result_file = execute_prompt({
      'model': model,
      'prompt-name': prompt_name,
      'prompt-version': prompt_version,
      'loglevel': 0,
      'stop-after-step': 3,
      'proj-name': proj_name,
      'input-filepath': 'text-output',
      'input-filename': '01-paraphr-short-headlines_gpt-3.md',
      'folders': folders,
      'prompt-obj': prompt_obj,
      'language': 'lang-de',
      'length-type': 'length-max-dyn',
      'length-max': 0.5,
      'toc-text': '',
      'result-filename': '02-paraphr-short.md'
  })

#step_1()
#step_2()
#step_3()

# Idea: chain by returning filename of last file => can be input for next

In [ ]:
notiz = {
  "prompt": "I want you to act as a very proficient SEO and high-end. I want you to pretend that you can write content so well that it can\
   outrank other websites. All output must be 100% human writing style and fix grammar issues and change to active voice. Your task is to\
   write an article starting with SEO Title with a bold letter and rewrite the content and include subheadings using related keywords.\
   Write in the style and tone of a professional IT magazine for software experts. Write every sentence in your very own words and do not\
   only rephrase the given content. The article must be 100% unique and remove plagiarism. Do not write a summary or conclusions in the\
  last paragraph. Avoid lists. Include subheadings. {language_text} {length_text} Text:```{input_text}```"
}

In [ ]:
proj_name = 'Handelsblatt'
#create_file_directory(proj_name + '/' + folders['text-output'], True)
#create_file_directory(proj_name + '/' + folders['text-output-logs'], True)
#create_test_project(proj_name, '')


# Step 1
def step_1():
  model = 'gpt-4'
  prompt_obj = get_prompt_from_lib(promptlib_work, 'summary-multiple-articles', model, '1')
  #print(json.dumps(prompt_obj, sort_keys=False, indent=2))
  result_text = execute_prompt({
      'model': model,
      'loglevel': 0,
      'stop-after-step': 0,
      'proj-name': proj_name,
      'input-filepath': '', #'text-output',
      'input-filename': '', #'paraphrased.txt',
      'folders': folders,
      'prompt-obj': prompt_obj,
      'language': 'lang-de',
      'length-type': '',
      'length-max': 0,
      'toc-text': '',
      'result-filename': '01-paraphr-short.md'
  })

#step_1()

# Current Work / Test

In [ ]:
promptlib_global['summary-shortened-same-tone'] = {
      'description': 'Abgeleitet aus "summary-shortened"',
      'category': 'summarize',
      '1': {
          'note': 'Summary without changing the tone',
          "prompt": "Paraphrase the following text and shorten each paragraph of the paraphrased text to maxiumum the half of its original length by focussing\
            on the key aspects. Write in the same style and tone as the given text. Be detailed and precise\
            and use every content detail of the text. {language_text} {length_text} Text:```{input_text}```",
          "lang-de": "Write everything in German language.",
          "lang-en": "Write everything in English language.",
          "lang-same": "If the following text in triple backticks is in German language write your text in German language, too.",
          'gpt-3.5-turbo': {
              "input-text-max": 1200,
              "length-max-fix": "The length of the full text must be 1200 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'gpt-4': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the full text must be 2400 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          }
      }
    }
text_from_file = "<File:>"
comment = 'Kommentar zum Projekt...'
# execute_project('S4HANA', 'gpt-4', promptlib_global, 'summary-shortened-same-tone', '1', 'length-max-dyn', 0, '', True, folders, text_from_file, comment)

In [ ]:
comment = 'Kommentar zum Projekt...'
#execute_project('Handelsblatt Fr', 'gpt-4', promptlib_global, 'summary-multiple-articles', '1', 'length-max-dyn', 5, '' ,True, folders, '<File:>', comment)

In [ ]:
promptlib_global['article-create-toc'] = {
      'description': '',
      'category': 'create-article',
      '1': {
          'note': 'Step 1: Create TOC',
          "prompt": "Here is some context for an article. The context is marked with 'Text:' and delimited by triple backticks. \
            Create a table of contents for an article titled 'Der Wert von Geschriebenem in Zeiten von Chat-GPT'. {language_text} {length_text} Text:```{input_text}``` {toc_text}",
          "lang-de": "Write everything in German language.",
          "lang-en": "Write everything in English language.",
          "lang-same": "If the following text in triple backticks is in German language write your text in German language, too.",
          'gpt-3.5-turbo': {
              "input-text-max": 1200,
              "length-max-fix": "The length of the full text must be 1200 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'gpt-4': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the full text must be 2400 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'default-model': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the full text must be 2400 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          }
      }
    }
promptlib_global['article-create-chapter'] = {
      'description': '',
      'category': 'create-article',
      '1': {
          'note': 'Step 2: Create first chapter',
          "prompt": "Here is some context for an article titled 'Der Wert von Geschriebenem in Zeiten von Chat-GPT'. The context is marked with 'Text:' and delimited by triple backticks. \
            The table of contents is marked with 'TOC:' and delimited by triple backticks.\
            Write chapter 1 with a length of 500 words. {language_text} {length_text} Text:```{input_text}``` {toc_text}",
          "lang-de": "Write everything in German language.",
          "lang-en": "Write everything in English language.",
          "lang-same": "If the following text in triple backticks is in German language write your text in German language, too.",
          'gpt-3.5-turbo': {
              "input-text-max": 1200,
              "length-max-fix": "The length of the full text must be 1200 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'gpt-4': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the full text must be 2400 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'default-model': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the full text must be 2400 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          }
      }
    }
text_from_file = "<File:>"

# Step 1
comment = 'Kommentar zum Projekt...'
#execute_project('article-future-of-text', 'gpt-4', promptlib_global, 'article-create-toc', '1', '', 0, '', True, folders, text_from_file, comment)

# Step 2
toc_text = f"""TOC: \`\`\`Inhaltsverzeichnis:
Titel: Der Wert von Geschriebenem in Zeiten von Chat-GPT
1. Künstliche Intelligenz und ihre Auswirkungen auf die täglichen Aufgaben
2. Die Rolle von Bots in der schriftlichen Kommunikation
3. Verhandlungen und Diskussionen in einer von Bots dominierten Welt
4. Die Auswirkungen von Künstlicher Intelligenz auf die Kreativität und die Wahrnehmung von Originalität
5. Die Auswirkungen von Künstlicher Intelligenz auf die Arbeit von Autoren und die Bedeutung von Charisma
6. Fazit: Die Zukunft von Geschriebenem in Zeiten von Chat-GPT.
\`\`\`"""
#execute_project('article-future-of-text', 'gpt-4', promptlib_global, 'article-create-chapter', '1', '', 0, toc_text, False, folders, text_from_file, comment)




In [ ]:
#import_prompt_lib()
#import_helper_lib()
#print(test_pypandoc())

# Ideas
- Markdown Konvertierung wenn docx
- Chaining (ausgabefile)
- Artikel schreiben TOC, danach Kapitel


## How to Overcome GPT Token Limit: With Vector indices
- Vector search enables the inclusion of only pertinent information from the loaded data into the model's context.
- Steps to implement this solution include registering for a database service, splitting documents into smaller chunks, converting them into vectors, and querying the index to find relevant information.
- The provided code examples utilize the Pinecone database and OpenAI's Ada model for vector conversion, but other alternatives can be explored.
- https://uxplanet.org/how-to-overcome-gpt-token-limit-721c30a18d55

# Quick Action

In [ ]:
#import_promptlib() # during dev: update promptlib from promptlib.ipynb
my_input_text = widgets.Textarea(placeholder='Enter input text', layout=Layout(width='50%', height='150px'), disabled=False)
button = widgets.Button(description="Summarize!")
def on_button_clicked(b):
  comment = 'Kommentar zum Projekt...'
  #execute_project('Handelsblatt', 'gpt-3.5-turbo', '', promptlib_global, 'summary-multiple-articles', '1', 'length-max-dyn', '', 0 ,True, folders, my_input_text.value, comment)
  execute_project('quick-action', 'gpt-3.5-turbo', 'Shorten this text, answer in German: ', {}, 'Manual prompt', '', '1000', '', 0, True, folders, my_input_text.value, comment)
button.on_click(on_button_clicked)
display(my_input_text, button)

# Manual Prompt

- Variable `input_dev` enthält Kontext
- Variable `prompt_dev` enthält Prompt

**Aufruf:**
```
execute_project('quick-action', 'gpt-3.5-turbo', prompt_dev, {}, 'Manual prompt', '', '500', '', 2, True, folders, input_dev, comment)
```
- quick-action: Projekt/Ordnername
- '500' Anzahl Wort-Split bei langen Texten
- 2 => Abbruch nach Step 2 (für Dev), sonst 0


In [ ]:
# @title
input_dev = f"""
Hello World
"""

In [ ]:
prompt_dev = f"""
Du bist mein persönlicher Assistent, der mir bei der Tages- und Wochenplanung hilft. Ich übermittle nun meine Notizen und Aufzeichnungen. Es ist in chronologisch umgekehrter Reihenfolge aufgebaut, die neuesten Einträge stehen zu Beginn. Die Tage sind in Markdown-Syntax  mit "# " gekennzeichnet, die Themenabschnitte mit "## ". Markierungen für Aufgaben oder Tags können an beliebigen Stellen in den Notizen auftauchen: Offene Aufgaben sind immer mit "[ ]" markiert, erledigte Aufgaben mit "[x]".
Bitte beantworte alle Folgefragen nur und ausschließlich auf Basis der übermittelten Aufzeichnungen.
Gebe alle Texte 1:1 wieder und erfinde nichts neues.
Erstelle eine Liste aller offenen Aufgaben als kurze bullet-point Liste.

Hier die Notizen:
"""


In [ ]:
prompt_summarize1 = f"""
You are in the role of an assistant for an editor.
Shorten the following text. Paraphrase the text, do not invent anything new and keep the original tone and style.
The output format is in Markdown syntax.
Write in German language.
"""
# Create 2 subheadings.

In [ ]:
prompt_summarize2 = f"""
You are in the role of an assistant for an editor.
Summarize the follwoing text in bulletpoints.
The output format is in Markdown syntax.
Write in German language.
"""
#Summarize the follwoing text in bulletpoints and group them with 2 subheadings.

# Wie wirkt sich Chunk-Größe auf Zusammenfassungsgröße in % aus?

 Chunkgröße | 500 | 750 | 1000 | 1500 | 2000
-------- | -------- | -------- | -------- | -------- | --------
Klima   | 85%   | 78% | 76% | x | 39%
A12 Model DIff   | 88%   | 85% | 87% | x |  38%
- mit GPT4:  | 67%   | 63% | 55% | 50% |  35%
Deloitte Podcast   | 50%   | 28%   | 21%  | x |  16%
B2B Content Podcast | 63%   | 43%  | 43%  | 19% |  17%
- mit GPT4: | 39%   | 31%  | 28%  | 23% |  10%
Adrenalin   | 89%   | 77%   | 77%  | x |  21%
Inhalt   | Inhalt   | Inhalt   | Inhalt  | Inhalt |  Inhalt

# Rules


- Default: 1000
- Sehr dichte Information (Fachtext): 2000
- Gesprochen, Podcast: 500
- GPT4 ist immer ca. 25% kürzer
- Zwischenüberschriften einfügen verändert/kürzt die Inhalte stark. Sollte man erst nachträglich / ganz am Schluss machen

# Rekursiver Test

 Step | Input words | Chunk-Size | 1 | 2 | 3 | 4 | Output words
-------- | -------- | -------- | -------- | -------- | -------- | -------- | --------  
A12 Model DIff | 7900 |1000   | 88% | 49% | 76% | - | 2500
A12 Model DIff | 7900 |1500   | 58% | 23% | 70% | - |753
Deloitte Podcast | 6600 |2000   | 17% | 47% | 100% | 100% | 546
Deloitte Podcast | 6600 |1000   | 24% | 81% | 99% | 100% | 1300
Deloitte Podcast | 6600 |500   | 51% | 96% | 77% | 98% | 2497


In [ ]:
create_file_directory('project', True)

step1 = execute_project('project/step1', 'gpt-3.5-turbo', prompt_summarize1, {}, 'Manual prompt', '', '1000', '', 0, True, folders, input_dev, comment)

step2 = execute_project('project/step2', 'gpt-3.5-turbo', prompt_summarize1, {}, 'Manual prompt', '', '1000', '', 0, True, folders, step1[2], comment)

step3 = execute_project('project/step3', 'gpt-3.5-turbo', prompt_summarize1, {}, 'Manual prompt', '', '1000', '', 0, True, folders, step2[2], comment)

bullet1 = execute_project('project/bullet1', 'gpt-3.5-turbo', prompt_summarize2, {}, 'Manual prompt', '', '1000', '', 0, True, folders, input_dev, comment)

bullet2 = execute_project('project/bullet2', 'gpt-3.5-turbo', prompt_summarize2, {}, 'Manual prompt', '', '1000', '', 0, True, folders, bullet1[2], comment)

bullet3 = execute_project('project/bullet3', 'gpt-3.5-turbo', prompt_summarize2, {}, 'Manual prompt', '', '1000', '', 0, True, folders, bullet2[2], comment)

zip_full_archive("./project/", "./project_full.zip")

In [ ]:
# Du bist in der Rolle eines Business Analysten in einer Beratungsfirma. Fasse die wichtigsten Punkte des folgenden Textes zusammen: